In [5]:
cd kaggle

/Users/martiom/kaggle


In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn import preprocessing as pre
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.calibration import CalibratedClassifierCV


In [81]:
train = pd.read_csv('train.csv')
train.replace([' ', '', -1], np.nan, inplace = True)

Load data
Extract labels
Remove labels and quote number
    Convert date to datetime

convert date.
extract year, month, day, weekday from date and drop date

In [154]:
def convert_date_data(date_data):
    date = pd.to_datetime(pd.Series(date_data))
    return np.vstack([date.dt.year, date.dt.month, 
                      date.dt.day, date.dt.dayofweek])

from collections import Counter
def extract_object_data(row):
    counts = Counter(row)
    row = row.fillna(max(filter(lambda x: x != np.nan, counts.keys()), 
               key = lambda x: counts[x]))
    data = pd.get_dummies(row).values
    if data.shape[1] == 1:
        print 'constant data'
        return []
    return data.T

def extract_numeric_data(row):
    row = row.fillna(row.median())
    if len(row.unique()) == 1:
        return []

    row = row - row.mean()
    row = row/row.std()
    return row.values

def extract_ordinal_data(row):
    return extract_numeric_data(row)



In [165]:

def convert_data(dataframe):
    y = dataframe.QuoteConversion_Flag.values
    X = []
    X.append(convert_date_data(dataframe['Original_Quote_Date']))
    columns_to_skip = ['Original_Quote_Date', 'QuoteConversion_Flag',
                      'QuoteNumber']
    for c in dataframe.columns:
        if c in columns_to_skip:
            print c, 'skipped'
            continue
        if train[c].dtype == object:
            X.append(extract_object_data(dataframe[c]))
        elif c.endswith('A') or c.endswith('B'):
            X.append(extract_ordinal_data(dataframe[c]))
        else:
            X.append(extract_numeric_data(dataframe[c]))
    X = filter(lambda x: len(x) > 0, X)
    return np.vstack(X).T, y


In [166]:
X, y = convert_data(train)

QuoteNumber skipped
Original_Quote_Date skipped
QuoteConversion_Flag skipped


- label encoder vs one hot encoder
- median calculation
- data normalization (without median)

In [9]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.20, random_state=36)


In [6]:
X, y = np.load('train_X.npy'), np.load('train_y.npy')

In [7]:
def check_classifier(Xtrain, ytrain, Xtest, ytest, clf):
    clf.fit(Xtrain, ytrain)
    print clf.score(Xtrain, ytrain)
    print clf.score(Xtest, ytest)

In [176]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf_isotonic = CalibratedClassifierCV(clf, cv=5, method='isotonic')
check_classifier(Xtrain, ytrain, Xtest, ytest,clf_isotonic )

0.917019012282
0.916204866637


In [177]:

clf = RandomForestClassifier(n_estimators=111, n_jobs=-1)
check_classifier(Xtrain, ytrain, Xtest, ytest,clf )

clf = RandomForestClassifier(n_estimators=111, n_jobs=-1)
clfbag = BaggingClassifier(clf, n_estimators=5)
check_classifier(Xtrain, ytrain, Xtest, ytest,clfbag )

clf = RandomForestClassifier(n_estimators=111, n_jobs=-1)
clf_isotonic = CalibratedClassifierCV(clf, cv=5, method='isotonic')
check_classifier(Xtrain, ytrain, Xtest, ytest,clf_isotonic )

0.993024994966
0.899292439263
0.9801200372
0.90690494909
0.998686493897
0.909397710495


In [178]:

clf = RandomForestClassifier(n_estimators=111, n_jobs=-1)
check_classifier(Xtrain, ytrain, Xtest, ytest,clf )

clf = RandomForestClassifier(n_estimators=111, n_jobs=-1)
clfbag = BaggingClassifier(clf, n_estimators=5)
check_classifier(Xtrain, ytrain, Xtest, ytest,clfbag )

clf = RandomForestClassifier(n_estimators=111, n_jobs=-1)
clf_isotonic = CalibratedClassifierCV(clf, cv=5, method='isotonic')
check_classifier(Xtrain, ytrain, Xtest, ytest,clf_isotonic )

1.0
0.910433165232
0.984261895859
0.909551111196
1.0
0.914958485935


In [182]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
check_classifier(Xtrain, ytrain, Xtest, ytest,clf )

clf = GaussianNB()
clfbag = BaggingClassifier(clf, n_estimators=5)
check_classifier(Xtrain, ytrain, Xtest, ytest,clfbag )

clf = GaussianNB()
clf_isotonic = CalibratedClassifierCV(clf, cv=5, method='isotonic')
check_classifier(Xtrain, ytrain, Xtest, ytest,clf_isotonic )

0.416597156307
0.413510766812
0.40460302394
0.400682633123
0.810346976539
0.812812793619


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from itertools import product
from sklearn.ensemble import VotingClassifier
dc = lambda: KNeighborsClassifier(7)

clf = dc()
check_classifier(Xtrain, ytrain, Xtest, ytest,clf )

clf = dc()
clfbag = BaggingClassifier(clf, n_estimators=5)
check_classifier(Xtrain, ytrain, Xtest, ytest,clfbag )

clf = dc()
clf_isotonic = CalibratedClassifierCV(clf, cv=5, method='isotonic')
check_classifier(Xtrain, ytrain, Xtest, ytest,clf_isotonic )

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import GaussianNB
clf1 = LogisticRegression(random_state=1)
clf1_isotonic = CalibratedClassifierCV(clf1, cv=5, method='isotonic')

clf2 = RandomForestClassifier(n_estimators = 111)
clf2_isotonic = CalibratedClassifierCV(clf2, cv=5, method='isotonic')

clf3 = GaussianNB()
clf3_isotonic = CalibratedClassifierCV(clf3, cv=5, method='isotonic')

eclf = VotingClassifier(estimators=[('lr', clf1_isotonic), 
                                    ('rf', clf2_isotonic), 
                                    ('gnb', clf3_isotonic)], voting='soft')
check_classifier(Xtrain, ytrain, Xtest, ytest,eclf )

In [ ]:
import sklearn
sklearn.__file__